In [1]:
### Script to integrate and align all available data sources:
# Single Cell RNA Seq
# Cytokine Data
# Neutrophil Data
# Proteomics

#############################################
# Prerequisites - Load Libraries

In [2]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/stark_stemi_R_Env_4_1//lib/R/library"



Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union



Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
   

In [3]:
source('MS4_Plot_Config.r')

Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”


###############################################
# Preqrequisites Configurations & Parameters

In [4]:
data_path =   '../data/current'

In [5]:
result_path = '../results/current'

In [6]:
### Should quantile normalization be applied?

In [7]:
quantile_normalization_cyto = FALSE

In [8]:
quantile_normalization_proteomics = FALSE

In [9]:
library_adjustment_neutrophils = TRUE

In [10]:
regress_neutrophils = FALSE

In [11]:
neutrophil_threshold = 0.2 # decide how many 0 are allowed on genes measured(percentage zeros)

In [12]:
quantile_normalization_neutrophils = TRUE

In [13]:
quantile_normalization_single_cell = TRUE

In [14]:
align_genes = FALSE  # decide whether to take only genes of single-cell data

In [15]:
add_duplicates = FALSE

In [16]:
# Name on which to Save the Data
name = 'V_FINAL'

# Functions

In [17]:
### Function for quantile normalization

quantile_normalization = function(X){
  ranks = apply(X, 2, rank, ties.method = 'min')  # determine ranks of each entry
  
  sorted = data.frame(apply(X, 2, sort)) # sort the entries
  means = apply(sorted, 1, mean) # calculate the means
  
  normalized_data = apply(ranks, 2 ,function(x){ means[x]}) # substitute the means into ranks matrix
}


In [18]:
### Gene wise quantile normalization


stdnorm <- function(x) {
  r = rank(x, ties.method="random")
  qnorm(r / (length(x) + 1))
}

# Load Data 

## Sample Meta Data

### Load

In [19]:
path = paste0(result_path, '/00_Data_Overview/Available_Data_per_Sample_Overview.csv')
all_samples_info = read.csv(path)
print(file.info(path)$mtime)

[1] "2023-12-10 11:16:00 CET"


In [20]:
path = paste0(result_path, '/00_Data_Overview/Merged_Sample_Meta_Data.csv')
sample_data = read.csv(path)
print(file.info(path)$mtime)

[1] "2023-12-10 11:16:00 CET"


In [21]:
#patients_filter = unique(sample_data$sample_id[is.na(str_extract(sample_data$sample_id, 'k'))])  # use only acs samples
patients_filter = unique(sample_data$sample_id) #  use all samples

### Process Clinical Data

In [22]:
### Select relevant columsn

In [23]:
clinical_data = sample_data[,c('sample_id', 'measurement', 'CK', 'CK_MB', 'Troponin','CRP', 'delta_ef_value', 'sample')]

In [24]:
clinical_data = clinical_data[clinical_data$sample_id %in% patients_filter,]

In [25]:
### Data transformations

In [26]:
clinical_data$CK_MB = as.numeric(clinical_data$CK_MB)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [27]:
clinical_data$CRP = as.numeric(clinical_data$CRP)

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [28]:
clinical_data[,3:6] = log2(clinical_data[,3:6]+1)   # logarithmize

In [29]:
clinical_data = clinical_data %>% group_by(sample_id) %>% summarise(CK = mean(CK), CK_MB = mean(CK_MB), Troponin = mean(Troponin), delta_ef_value = mean(delta_ef_value), CRP =mean(CRP) )

In [30]:
clinical_data = data.frame(clinical_data)

In [31]:
### Long format to integrate in clustering

In [32]:
clinical_data_long = melt(clinical_data)

Using sample_id as id variables



In [33]:
clinical_data_long$type = 'clinical_data'

In [34]:
clinical_data_dupl = clinical_data_long[clinical_data_long$sample_id %in% c('m13.2', 'm6.4'),]

In [35]:
clinical_data_dupl$sample_id[clinical_data_dupl$sample_id == 'm13.2'] = 'm13.22'

In [36]:
clinical_data_dupl$sample_id[clinical_data_dupl$sample_id == 'm6.4'] = 'm6.42'

In [37]:
if(add_duplicates == TRUE){
    clinical_data_long = rbind(clinical_data_long, clinical_data_dupl)
    }

In [38]:
unique(clinical_data_long$variable)

[1] CK             CK_MB          Troponin       delta_ef_value CRP           
Levels: CK CK_MB Troponin delta_ef_value CRP

## Cytokine Data

### Load

In [39]:
### Load processed cytokine data

In [40]:
path = paste0(result_path, '/00_Data_Overview/Prepared_Cytokine_Data.csv')
cytokines = read.csv( path)
print(file.info(path)$mtime)

[1] "2023-12-10 11:16:00 CET"


In [41]:
### Load cytokine gene mapping

In [42]:
path = paste0(data_path, '/preprocessed-data/meta-data/Cytokine_Gene_Mapping.csv')
cytokine_gene_mapping = read.csv( path)
print(file.info(path)$mtime)

[1] "2022-07-13 11:17:31 CEST"


In [43]:
head(cytokine_gene_mapping,2)

,cytokine,mapped_name
,<chr>,<chr>
1,IL8,IL8__CXCL8
2,MIP1alpha,MIP1alpha__CCL3


In [44]:
cytokine_gene_mapping[cytokine_gene_mapping$cytokine == 'IL15',]

,cytokine,mapped_name
,<chr>,<chr>
26,IL15,IL15__IL15


In [45]:
ncol(cytokines)  # about 75 cytokines

[1] 75

### Pre-process

In [46]:
#### Set OOR values to 0

In [47]:
cytokines[cytokines == 'OOR <'] = 0

In [48]:
cytokines[cytokines == 'OOR'] = 0

In [49]:
cytokines[cytokines == ''] = 0

In [50]:
rownames(cytokines) = cytokines$id

In [51]:
samples = cytokines$id

In [52]:
cytokines$id = NULL

In [53]:
cytokines$sample_id = NULL

In [54]:
cytokines$X = NULL

In [55]:
colnames(cytokines) = str_replace(colnames(cytokines), '\\.|\\.\\.|\\.\\.\\.', '_')

In [56]:
for(i in colnames(cytokines)){
    cytokines[,i] = as.numeric(   cytokines[,i])
    }

In [57]:
cytokine_names = colnames(cytokines)

In [58]:
cytokines_trans_adapted = cytokines

In [59]:
cytokines_trans_adapted = data.frame(cytokines_trans_adapted)

In [60]:
cytokines_trans_adapted$X = NULL

In [61]:
cytokines_trans_adapted$sample_id = NULL

In [62]:
#### Logarithmize the values (TBD - other transformations?)

In [63]:
cytokines_trans_adapted = log2(cytokines_trans_adapted + 1)

In [64]:
cytokines_trans_adapted$sample_id = samples

In [65]:
cytokines_trans_adapted$cytokine_data = NULL

In [66]:
#### Apply quantile normalization

In [67]:
quantile_normalization_cyto

[1] FALSE

In [68]:
if(quantile_normalization_cyto == TRUE){
    rownames(cytokines_trans_adapted) = cytokines_trans_adapted$sample_id
    cytokines_trans_adapted$sample_id = NULL
    cytokines_trans_adapted = data.frame(t(cytokines_trans_adapted))
    cyto_names = rownames(cytokines_trans_adapted)
    
    
    cytokines_trans_adapted = quantile_normalization(cytokines_trans_adapted)
    rownames(cytokines_trans_adapted) = cyto_names
    cytokines_trans_adapted = data.frame(t(cytokines_trans_adapted))
    cytokines_trans_adapted$sample_id = rownames(cytokines_trans_adapted)
    }

In [69]:
#### Generate cytokine long format for visualization

In [70]:
cytokines_trans_adapted$id = NULL

In [71]:
cytokines_trans_adapted$cytokine_data = NULL

In [72]:
cytokines_long = melt(cytokines_trans_adapted)

Using sample_id as id variables



In [73]:
cytokines_long$type = 'cytokine'

In [74]:
### Adjust names for later mapping

In [75]:
cytokines_long$variable = as.character(cytokines_long$variable)

In [76]:
cytokines_long = merge(cytokines_long, cytokine_gene_mapping, by.x = c('variable'), by.y = c('cytokine'), all.x = TRUE)

In [77]:
cytokines_long$mapped_name[is.na(cytokines_long$mapped_name)] = cytokines_long$variable[is.na(cytokines_long$mapped_name)]

In [78]:
cytokines_long$variable = cytokines_long$mapped_name

In [79]:
cytokines_long$mapped_name = NULL

In [80]:
cytokine_names

[1] "sCD40L"         "EGF"            "Eotaxin"        "FGF2"          
 [5] "FLT3L"          "Fractalkine"    "GCSF"           "GMCSF"         
 [9] "GROalpha"       "IFNalpha2"      "IFNgamma"       "IL1alpha"      
[13] "IL1beta"        "IL1RA"          "IL2"            "IL3"           
[17] "IL4"            "IL5"            "IL6"            "IL7"           
[21] "IL8"            "IL9"            "IL10"           "IL12p40"       
[25] "IL12p70"        "IL13"           "IL15"           "IL17A"         
[29] "IL17E_IL25"     "IL17F"          "IL18"           "IL22"          
[33] "IL27"           "IP10"           "MCP1"           "MCP3"          
[37] "MCSF"           "MDC"            "MIG_CXCL9"      "MIP1alpha"     
[41] "MIP1beta"       "PDGFAA"         "PDGFAB_BB"      "RANTES"        
[45] "TGFalpha"       "TNFalpha"       "TNFbeta"        "VEGFA"         
[49] "X6CKine"        "BCA1"           "CTACK"          "ENA78"         
[53] "Eotaxin2"       "Eotaxin3"       "I309"           "IL16"          
[57] "IL20"           "IL21"           "IL23"           "IL28A"         
[61] "IL33"           "LIF"            "MCP2"           "MCP4"          
[65] "MIP1_"          "SCF"            "SDF1alpha_beta" "TARC"          
[69] "TPO"            "TRAIL"          "TSLP"           "cytokine_data"

In [81]:
cytokine_names = unique(cytokines_long$variable)

In [82]:
### Add duplicates

In [83]:
cytokines_long_dupl = cytokines_long[cytokines_long$sample_id %in% c('m13.2', 'm6.4'),]

In [84]:
cytokines_long_dupl$sample_id[cytokines_long_dupl$sample_id == 'm13.2'] = 'm13.22'

In [85]:
cytokines_long_dupl$sample_id[cytokines_long_dupl$sample_id == 'm6.4'] = 'm6.42'

In [86]:
if(add_duplicates == TRUE){
    cytokines_long = rbind(cytokines_long, cytokines_long_dupl)
    }

In [87]:
length(unique(cytokines_long$variable))

[1] 71

## Proteomic Data

### Load

In [88]:
path = paste0(result_path, '/00_Data_Overview/Prepared_Proteomic_Data.csv')
proteomics = read.csv( path)
print(file.info(path)$mtime)

[1] "2023-12-10 11:16:09 CET"


### Pre-Process

In [89]:
rownames(proteomics) = proteomics$X

In [90]:
proteomics$X = NULL

In [91]:
proteomics$proteomics_data = NULL

In [92]:
ncol(proteomics)  # about 490 proteins measured

[1] 491

In [93]:
head(sort(colnames(proteomics)))

[1] "A1BG_M0R009"   "A1BG_P04217.2" "A2M_P01023"    "ABI3BP_D3YTG3"
[5] "ACTB_P60709"   "ACTC1_P68032"

In [94]:
proteomic_names = colnames(proteomics)

#### Adjust distribution

In [95]:
#colMeans(proteomics)

In [96]:
quantile_normalization_proteomics

[1] FALSE

In [97]:
if (quantile_normalization_proteomics == TRUE){
    rownames(proteomics) = proteomics$sample_id
    proteomics$sample_id = NULL
    proteomics = t(proteomics)
    names = rownames(proteomics)
    
    proteomics  = quantile_normalization(proteomics )  # works on proteomics data
    rownames(proteomics) = names
    proteomics = data.frame(t(proteomics))
    proteomics$sample_id = rownames(proteomics)
    }

#### Prepare long format

In [98]:
proteomics_long =  melt(proteomics)

Using sample_id as id variables



In [99]:
proteomics_long$type = 'proteomics'

In [100]:
### Add dupl

In [101]:
proteomics_long_dupl = proteomics_long[proteomics_long$sample_id %in% c('m13.2', 'm6.4'),]

In [102]:
proteomics_long_dupl$sample_id[proteomics_long_dupl$sample_id == 'm13.2'] = 'm13.22'

In [103]:
proteomics_long_dupl$sample_id[proteomics_long_dupl$sample_id == 'm6.4'] = 'm6.42'

In [104]:
if(add_duplicates == TRUE){
    proteomics_long = rbind(proteomics_long, proteomics_long_dupl)
    }

In [105]:
length(unique(proteomics_long$variable))

[1] 490

## Neutrophil Data

### Load

In [106]:
path = paste0(result_path, '/00_Data_Overview/Prepared_Neutrophil_Data.csv')
neutrophils = read.csv(path)
print(file.info(path)$mtime)

[1] "2023-12-10 11:16:08 CET"


In [107]:
nrow(neutrophils)

[1] 121

### Pre-Process

#### Adjust gene-names

In [108]:
rownames(neutrophils) = neutrophils$sample_id

In [109]:
neutrophils$X = NULL

In [110]:
neutrophils$sample_id = NULL

In [111]:
neutrophils$neutrophil_data = NULL

In [112]:
summary(rowSums(neutrophils))

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   1840   12677   19266   24080   27994  105324 

In [113]:
genes = colnames(neutrophils)[!is.na(str_extract(colnames(neutrophils), 'ENSG'))]

In [114]:
length(unique(genes))

[1] 38146

In [115]:
length(genes)

[1] 38146

In [116]:
tail(genes)

[1] "ENSG00000288617.1" "ENSG00000288621.1" "ENSG00000288631.1"
[4] "ENSG00000288637.1" "ENSG00000288638.1" "ENSG00000288642.1"

In [117]:
genes = str_replace(genes, '\\..*', '') ## Adjust format for mapping

In [118]:
table(genes)[table(genes) > 1] # find duplicates caused by transformation and remove (TBD: check whether really need to be removed!)

genes
ENSG00000002586 ENSG00000124333 ENSG00000167393 ENSG00000169084 ENSG00000169093 
              2               2               2               2               2 
ENSG00000169100 ENSG00000182162 ENSG00000182378 ENSG00000182484 ENSG00000185291 
              2               2               2               2               2 
ENSG00000185960 ENSG00000196433 ENSG00000197976 ENSG00000198223 ENSG00000214717 
              2               2               2               2               2 
ENSG00000226179 ENSG00000236017 
              2               2 

In [119]:
colnames(neutrophils)[!is.na(str_extract(colnames(neutrophils), 'ENSG00000002586'))]  # example for duplicate!

[1] "ENSG00000002586.20"       "ENSG00000002586.20_PAR_Y"

In [120]:
genes = genes[! genes %in% names(table(genes)[table(genes) > 1])]

In [121]:
length(genes)

[1] 38112

In [122]:
length(unique(genes))

[1] 38112

In [123]:
length(unique(genes))

[1] 38112

In [124]:
### Map genes to SYMBOL

In [125]:
genes_mapped = bitr(genes, fromType="ENSEMBL", toType="SYMBOL", OrgDb = 'org.Hs.eg.db') ### Map genes to SYMBOL

'select()' returned 1:many mapping between keys and columns

Warning message in bitr(genes, fromType = "ENSEMBL", toType = "SYMBOL", OrgDb = "org.Hs.eg.db"):
“33.75% of input gene IDs are fail to map...”


In [126]:
head(genes_mapped,2)

,ENSEMBL,SYMBOL
,<chr>,<chr>
1,ENSG00000000003,TSPAN6
2,ENSG00000000419,DPM1


In [127]:
nrow(genes_mapped)  # 25.338 genes that can be mapped to SYMBOL!

[1] 25443

In [128]:
'CX3CL1' %in% genes_mapped$SYMBOL  # This is the gene coding for cytokine protein: Fractalkine - see: https://www.uniprot.org/uniprot/P78423

[1] TRUE

In [129]:
'SERPINA1'  %in% genes_mapped$SYMBOL

[1] TRUE

In [130]:
### Adjust neutrophil gene names

In [131]:
neutrophils$neutrophil_data = NULL

In [132]:
neutrophils$sample_id = NULL

In [133]:
neutrophils = data.frame(t(neutrophils))

In [134]:
neutrophils$gene = rownames(neutrophils)

In [135]:
neutrophils$gene = str_replace(neutrophils$gene, '\\..*', '')

In [136]:
head(genes_mapped, 2)

,ENSEMBL,SYMBOL
,<chr>,<chr>
1,ENSG00000000003,TSPAN6
2,ENSG00000000419,DPM1


In [137]:
neutrophils = merge(neutrophils, genes_mapped, by.x = 'gene', by.y = 'ENSEMBL')

In [138]:
nrow(neutrophils)

[1] 25443

In [139]:
length(unique(neutrophils$SYMBOL))

[1] 25378

In [140]:
neutrophils = neutrophils %>% group_by(SYMBOL) %>% summarise(across(-gene, \(x) sum(x, na.rm = TRUE)))

In [141]:
neutrophils = data.frame(neutrophils)

In [142]:
nrow(neutrophils)

[1] 25378

In [143]:
rownames(neutrophils) = neutrophils$SYMBOL

In [144]:
neutrophils$SYMBOL = NULL

In [145]:
neutrophils = data.frame(t(neutrophils))

In [146]:
neutrophils$sample_id = rownames(neutrophils)

In [147]:
neutrophils$sample_id = NULL

In [148]:
neutrophil_names = colnames(neutrophils)

In [149]:
neutrophil_evaluation =  neutrophils

In [150]:
#colMeans(neutrophils)

#### Filter out ribosomal and mitochondrial genes

In [151]:
dim(neutrophils)

[1]   121 25378

In [152]:
neutrophils = neutrophils[,is.na(str_extract(colnames(neutrophils), '^MT.*|^RPL.*|^RPS.*'))]

In [153]:
dim(neutrophils)

[1]   121 24151

#### Investigate and Adjust distribution of values

In [154]:
### Check out distribution per sample

In [155]:
neutrophils$sample_id = rownames(neutrophils)

#### Filter out low expressed genes

In [156]:
### genes expressed in less than 50% of samples

In [157]:
#head(neutrophils == 0)

In [158]:
neutrophils$sample_id = NULL

In [159]:
nrow(neutrophils)

[1] 121

In [160]:
dim(neutrophils)

[1]   121 24151

In [161]:
neutrophil_threshold

[1] 0.2

In [162]:
ncol(neutrophils)

[1] 24151

In [163]:
nrow(neutrophils)

[1] 121

In [164]:
neutrophils = neutrophils[,((colSums(neutrophils == 0))/ nrow(neutrophils)) <= neutrophil_threshold]

In [165]:
ncol(neutrophils)

[1] 1190

In [166]:
### Check out distribution per sample

In [167]:
neutrophils$sample_id = rownames(neutrophils)

### Optional: remove samples with high amount of 0

In [168]:
sample_perc_zero = rowSums(neutrophils==0)/ ncol(neutrophils) 

In [169]:
head(sample_perc_zero,2)

k4      m14.3 
0.12762385 0.06633081

In [170]:
remove_samples = sample_perc_zero[sample_perc_zero > 0.1]

In [171]:
sort(names(remove_samples))

[1] "k15"   "k18"   "k2"    "k21"   "k23"   "k24"   "k27"   "k29"   "k31"  
[10] "k4"    "k9"    "m11.1" "m12.1" "m13.2" "m14.2" "m18.2" "m19.3" "m2.3" 
[19] "m2.4"  "m20.1" "m20.3" "m22.3" "m26.3" "m5.2"  "m5.4"  "m6.1"  "m6.2" 
[28] "m9.1"  "m9.2"

In [172]:
length(names(remove_samples))

[1] 29

In [173]:
neutrophils = neutrophils[!rownames(neutrophils) %in% names(remove_samples),]

In [174]:
length(unique(rownames(neutrophils)))

[1] 92

### Normalization

In [175]:
### Adjust for library size (10.000 counts per sample)

In [176]:
dim(neutrophils)

[1]   92 1191

In [177]:
neutrophils$sample_id =  NULL

In [178]:
scaling_factor = rowSums(neutrophils) /mean(rowSums(neutrophils))

In [179]:
head(scaling_factor)

m14.3     m26.2     m14.4      m3.3        k1     m24.4 
0.6323375 1.5346160 0.7127839 1.6074439 0.6597125 1.3365344

In [180]:
mean(rowSums(neutrophils))

[1] 15488.57

In [181]:
if(library_adjustment_neutrophils == TRUE){
    neutrophils = apply(neutrophils,2, function(x){ x/scaling_factor})
    }

In [182]:
head(rowSums(neutrophils) ) # check out counts per sample  - TBD compare with pseudobulk RNA- DATA

m14.3    m26.2    m14.4     m3.3       k1    m24.4 
15488.57 15488.57 15488.57 15488.57 15488.57 15488.57

In [183]:
#### logarithmize neutrophil data (TBD)
neutrophils = data.frame(log2(neutrophils + 1))

In [184]:
neutrophils = data.frame(neutrophils)

In [185]:
### Inspect variance

In [186]:
variance = apply(neutrophils, 2, var)

In [187]:
head(variance)

AATK     ABCA7     ABHD2     ABHD5     ABTB1     ACAP2 
0.8524371 0.8210484 0.5840491 0.5867826 0.5833892 0.5185382

In [188]:
mean(apply(neutrophils, 2, var))

[1] 0.6863863

In [189]:
var_threshold = quantile(variance, probs = seq(0, 1, 0.01), na.rm = FALSE,
         names = TRUE)['25%']

In [190]:
var_threshold

25% 
0.449859

In [191]:
length(variance)

[1] 1190

In [192]:
keep_genes = names(variance[variance > var_threshold])

In [193]:
ncol(neutrophils)

[1] 1190

In [194]:
neutrophils = neutrophils[, keep_genes]

In [195]:
ncol(neutrophils)

[1] 892

In [196]:
#### Regress out effect of mean expression + amount zero 

In [197]:
neutrophils$sample_id = rownames(neutrophils)

In [198]:
neutrophils_long = melt(neutrophils)

Using sample_id as id variables



In [199]:
summ_stats = neutrophils_long %>% group_by(sample_id) %>% summarise(sum_counts = mean(value), amount_zero = mean(value ==0))

In [200]:
head(summ_stats,2)

sample_id,sum_counts,amount_zero
<chr>,<dbl>,<dbl>
k1,2.374504,0.08520179
k10,2.446978,0.03699552


In [201]:
neutrophils = neutrophils_long %>% dcast(sample_id ~variable, value.var = "value")

In [202]:
rownames(neutrophils) = neutrophils$sample_id

In [203]:
### Check out distribution per sample

In [204]:
neutrophils$sample_id = rownames(neutrophils)

In [205]:
### Apply quantile normalization

In [206]:
neutrophils$sample_id = NULL

In [207]:
quantile_normalization_neutrophils

[1] TRUE

In [208]:
if(quantile_normalization_neutrophils  == TRUE){
    neutrophils = t(neutrophils)
    genes_neutrophils = rownames(neutrophils)
    neutrophils  = quantile_normalization(neutrophils ) 
    rownames(neutrophils) = genes_neutrophils
    neutrophils = data.frame(t(neutrophils))
    }

In [209]:
### Check out distribution per sample

In [210]:
neutrophils$sample_id = rownames(neutrophils)

### Filter only genes that are also in single cell

In [211]:
path = paste0(result_path, '/C-Analysis/C0_Filter_Genes_Input_Correlations_Perc_Values.csv')
genes_subset = read.csv(path) # cluster alternative
print(file.info(path)$mtime)
#genes_subset = read.csv(paste0(result_path, '/C-Analysis/C0_Filter_Genes_Input_Correlations_Perc_Values_cell_type_Scanorama.csv'))   # cell-type alternative

[1] "2024-01-03 14:10:30 CET"


In [212]:
head(genes_subset,2)

,X,perc_cells_expressing_gene,total_amount_cells_expressing_gene,gene,cluster
,<chr>,<dbl>,<int>,<chr>,<chr>
1,AL627309.1,0.24327612,18,AL627309.1,8_B-cell
2,AL627309.4,0.05406136,4,AL627309.4,8_B-cell


In [213]:
genes = unique(genes_subset$gene)

In [214]:
align_genes

[1] FALSE

In [215]:
if(align_genes == TRUE){
    neutrophils = neutrophils[,colnames(neutrophils) %in% genes]
    }

In [216]:
dim(neutrophils)

[1]  92 893

In [217]:
### Check out distribution per sample

In [218]:
neutrophils$sample_id = rownames(neutrophils)

### Prepare long format

In [219]:
neutrophils$sample_id = rownames(neutrophils)

In [220]:
neutrophils_long = melt(neutrophils)

Using sample_id as id variables



In [221]:
neutrophils_long$type = 'neutrophil'

In [222]:
head(neutrophils_long,2)

,sample_id,variable,value,type
,<chr>,<fct>,<dbl>,<chr>
1,k1,AATK,2.746071,neutrophil
2,k10,AATK,3.074688,neutrophil


In [223]:
### Add duplicates

In [224]:
neutrophils_long_dupl = neutrophils_long[neutrophils_long$sample_id %in% c('m13.2', 'm6.4'),]

In [225]:
neutrophils_long_dupl$sample_id[neutrophils_long_dupl$sample_id == 'm13.2'] = 'm13.22'

In [226]:
neutrophils_long_dupl$sample_id[neutrophils_long_dupl$sample_id == 'm6.4'] = 'm6.42'

In [227]:
head(neutrophils_long_dupl,2)

,sample_id,variable,value,type
,<chr>,<fct>,<dbl>,<chr>
85,m6.42,AATK,1.162339,neutrophil
177,m6.42,ABCA7,1.919833,neutrophil


In [228]:
if(add_duplicates == TRUE){
    neutrophils_long = rbind(neutrophils_long, neutrophils_long_dupl)
}

In [229]:
length(unique(neutrophils_long$variable))

[1] 892

In [230]:
length(unique(neutrophils_long$sample_id))

[1] 92

## RNA-Single-Seq

### Load cell-expression- gene cluster info

In [231]:
path = paste0(result_path, '/C-Analysis/C0_Filter_Genes_Input_Correlations_Perc_Values.csv')
cell_perc_cluster = read.csv( path) # cluster alternative
print(path)
print(file.info(path)$mtime)

[1] "../results/current/C-Analysis/C0_Filter_Genes_Input_Correlations_Perc_Values.csv"
[1] "2024-01-03 14:10:30 CET"


In [232]:
head(cell_perc_cluster,2)

,X,perc_cells_expressing_gene,total_amount_cells_expressing_gene,gene,cluster
,<chr>,<dbl>,<int>,<chr>,<chr>
1,AL627309.1,0.24327612,18,AL627309.1,8_B-cell
2,AL627309.4,0.05406136,4,AL627309.4,8_B-cell


In [233]:
nrow(cell_perc_cluster)

[1] 365199

In [234]:
length(unique(cell_perc_cluster$gene))

[1] 19221

In [235]:
length(unique(cell_perc_cluster$cluster))

[1] 19

In [236]:
cell_perc_cluster[((cell_perc_cluster$perc_cells > 10) & (cell_perc_cluster$total_amount_cells_expressing_gene > 1200))  ,] %>% group_by(cluster) %>% count()  # investigate amount of genes after filtering

cluster,n
<chr>,<int>
0_T-cell-CD4,3854
10_B-cell,1220
11_T-cell-CD4,981
12_Monocytes - CD16_FCGR3A,1039
13_Dendritic,406
14_Other,35
1_T-cell-CD8,4320
2_T-cell-CD4,3830
3_NK,3357


In [237]:
name

[1] "V_FINAL"

In [238]:
##### Decide on conditions for filtering genes out of single-cell data! (uncommented no filtering!)
if(name %in%  c('V_FINAL', 'V29')){
    cell_perc_cluster =  cell_perc_cluster[((cell_perc_cluster$perc_cells > 50) & (cell_perc_cluster$total_amount_cells_expressing_gene > 1200)) | ((cell_perc_cluster$perc_cells > 40) & (cell_perc_cluster$total_amount_cells_expressing_gene > 3000)) ,] 
    }
# condition removed for complete data

In [239]:
head(cell_perc_cluster,2)

,X,perc_cells_expressing_gene,total_amount_cells_expressing_gene,gene,cluster
,<chr>,<dbl>,<int>,<chr>,<chr>
95,RPL22,99.24314,7343,RPL22,8_B-cell
119,PARK7,45.38451,3358,PARK7,8_B-cell


In [240]:
nrow(cell_perc_cluster)

[1] 13206

### Load Pseudobulk aggregated RNA data from C0

In [241]:
name

[1] "V_FINAL"

In [242]:
if(name %in% c('V_FINAL')){
    path = paste0(result_path, '/C-Analysis/C0_aggregated_RNA_input_correlations_all.RDS')
    load(path)   
    print(path)
    print(file.info(path)$mtime)
    }


[1] "../results/current/C-Analysis/C0_aggregated_RNA_input_correlations_all.RDS"
[1] "2023-11-30 17:42:49 CET"


In [243]:
all_genes = rownames(pb)

In [244]:
head(all_genes)

[1] "AL627309.1" "AL627309.4" "AL669831.5" "FAM87B"     "LINC00115" 
[6] "FAM41C"

In [245]:
length(all_genes)

[1] 19221

In [246]:
pb

class: SingleCellExperiment 
dim: 19221 121 
metadata(2): experiment_info agg_pars
assays(19): 0_T-cell-CD4 1_T-cell-CD8 ... 8_B-cell 9_Monocytes -
  CD16_FCGR3A
rownames(19221): AL627309.1 AL627309.4 ... AC004556.1 AC240274.1
rowData names(0):
colnames(121): 1.1-L1 10-L11 ... 9.2-L4 9.3-L7
colData names(26): group_id classification_measurement ... library_char
  ident
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

In [247]:
head(rowSums(assay(pb, '0_T-cell-CD4')))

AL627309.1 AL627309.4 AL669831.5     FAM87B  LINC00115     FAM41C 
0.13504293 0.12041834 5.15396341 0.08123066 1.20562973 1.57305053

In [248]:
#pb_check = pb

In [249]:
assays(pb)

List of length 19
names(19): 0_T-cell-CD4 1_T-cell-CD8 ... 8_B-cell 9_Monocytes - CD16_FCGR3A

In [250]:
#unique(sample_data[,c('id', 'sample_id')])

In [251]:
sample_data$merge_id = paste0(sample_data$id, '-', sample_data$library)

In [252]:
colnames(sample_data)

[1] "X.1"                    "sample_id"              "sample"                
 [4] "id"                     "measurement"            "library"               
 [7] "id.y"                   "name"                   "read"                  
[10] "pattern"                "sequence"               "feature_type"          
[13] "X"                      "in_sample"              "measurement_tp"        
[16] "sc_rna_seq_data"        "age"                    "sex"                   
[19] "classification"         "group"                  "meta_data"             
[22] "delta_ef_value_group"   "delta_ef_value"         "delta_ef_value_class"  
[25] "ef_classification_data" "CK"                     "CK_MB"                 
[28] "Troponin"               "CRP"                    "clinical_data"         
[31] "merge_id"

### Pre-Process

#### Remove Clusters

In [253]:
### Filte pb on relevant clusters

In [254]:
pb

class: SingleCellExperiment 
dim: 19221 121 
metadata(2): experiment_info agg_pars
assays(19): 0_T-cell-CD4 1_T-cell-CD8 ... 8_B-cell 9_Monocytes -
  CD16_FCGR3A
rownames(19221): AL627309.1 AL627309.4 ... AC004556.1 AC240274.1
rowData names(0):
colnames(121): 1.1-L1 10-L11 ... 9.2-L4 9.3-L7
colData names(26): group_id classification_measurement ... library_char
  ident
reducedDimNames(0):
mainExpName: NULL
altExpNames(0):

In [255]:
names(assays(pb))

[1] "0_T-cell-CD4"               "1_T-cell-CD8"              
 [3] "10_B-cell"                  "11_T-cell-CD4"             
 [5] "12_Monocytes - CD16_FCGR3A" "13_Dendritic"              
 [7] "14_Other"                   "15_Plasma Blast"           
 [9] "16_Plasma Blast"            "17_Progenitor"             
[11] "18_Megakaryocytes"          "2_T-cell-CD4"              
[13] "3_NK"                       "4_Monocytes - CD14"        
[15] "5_T-cell-CD4"               "6_Monocytes - CD14"        
[17] "7_Monocytes - CD14"         "8_B-cell"                  
[19] "9_Monocytes - CD16_FCGR3A"

In [256]:
assay(pb, '18_Megakaryocytes') = NULL

In [257]:
assay(pb, 'Megakaryocytes') = NULL

In [258]:
assay(pb, '17_Progenitor') = NULL

In [259]:
assay(pb, 'Progenitor') = NULL

In [260]:
assay(pb, '16_Plasma Blast') = NULL

In [261]:
assay(pb, 'Plasma Blast') = NULL

In [262]:
assay(pb, '15_Plasma Blast') = NULL

In [263]:
assay(pb, 'Plasma Blast') = NULL

In [264]:
assay(pb, '14_Other') = NULL

In [265]:
assay(pb, 'Other') = NULL

In [266]:
length(names(assays(pb)))

[1] 14

#### Prepare gene-cluster dataframe + normalize

In [267]:
assays(pb)

List of length 14
names(14): 0_T-cell-CD4 1_T-cell-CD8 ... 8_B-cell 9_Monocytes - CD16_FCGR3A

In [268]:
#head(assay(pb, '1_T-cell-CD8'))

In [269]:
nodes = names(assays(pb))

In [270]:
head(nodes)

[1] "0_T-cell-CD4"               "1_T-cell-CD8"              
[3] "10_B-cell"                  "11_T-cell-CD4"             
[5] "12_Monocytes - CD16_FCGR3A" "13_Dendritic"

In [271]:
cell_types = nodes

In [272]:
#nodes = nodes[1:3]

In [273]:
final_data = data.frame(samples = colnames(pb))

In [274]:
final_data_vis = data.frame(samples = colnames(pb))

In [275]:
rownames(final_data) = final_data$samples

In [276]:
rownames(final_data_vis) = final_data_vis$samples

In [277]:
head(genes_subset,2)

,X,perc_cells_expressing_gene,total_amount_cells_expressing_gene,gene,cluster
,<chr>,<dbl>,<int>,<chr>,<chr>
1,AL627309.1,0.24327612,18,AL627309.1,8_B-cell
2,AL627309.4,0.05406136,4,AL627309.4,8_B-cell


In [278]:
nodes

[1] "0_T-cell-CD4"               "1_T-cell-CD8"              
 [3] "10_B-cell"                  "11_T-cell-CD4"             
 [5] "12_Monocytes - CD16_FCGR3A" "13_Dendritic"              
 [7] "2_T-cell-CD4"               "3_NK"                      
 [9] "4_Monocytes - CD14"         "5_T-cell-CD4"              
[11] "6_Monocytes - CD14"         "7_Monocytes - CD14"        
[13] "8_B-cell"                   "9_Monocytes - CD16_FCGR3A"

In [279]:
genes_subset$cluster = str_extract(genes_subset$cluster, paste0(nodes, collapse = '|'))

In [280]:
genes_subset = na.omit(genes_subset)

In [281]:
unique(genes_subset$cluster)

[1] "8_B-cell"                   "9_Monocytes - CD16_FCGR3A" 
 [3] "5_T-cell-CD4"               "0_T-cell-CD4"              
 [5] "1_T-cell-CD8"               "6_Monocytes - CD14"        
 [7] "2_T-cell-CD4"               "4_Monocytes - CD14"        
 [9] "11_T-cell-CD4"              "3_NK"                      
[11] "7_Monocytes - CD14"         "13_Dendritic"              
[13] "10_B-cell"                  "12_Monocytes - CD16_FCGR3A"

In [282]:
### Alternative define new gene_subset

In [283]:
genes_subset = cell_perc_cluster

In [284]:
is.na(str_extract(name, 'scano'))

[1] TRUE

In [285]:

for(i in nodes){
    data = assay(pb, i)


    ##### Normalize counts per sample (library size) - currently only for non-scanorama functions

    if(is.na(str_extract(name, 'scano')) == TRUE){
        scaling_factor = colSums(data) /mean(colSums(data))

        for (j in 1:ncol(data)){
            if(scaling_factor[j] != 0){
                data[,j] = data[,j]/ scaling_factor[j]
                }
            }
        }

    ### Subset data on genes with minimum expression in cluster
    data = data[rownames(data) %in% genes_subset$gene[genes_subset$cluster == i],]

    ### Alternative - cluster independent subsetting
    #data = data[rownames(data) %in% genes_subset,]

    ##### TBD pre-processing stepd

    if(is.na(str_extract(name, 'scano')) == TRUE){
        data = log2(data+1) # logarithmize count values (optional!)
        }

    #### Quantile normalization (TBD maybe also on complete dataset?)

    if(quantile_normalization_single_cell == TRUE){
        data_rows = rownames(data)
        data  = quantile_normalization(data ) 
        rownames(data) = data_rows
        }

    rownames(data) = paste0(i, '__' ,rownames(data))

    data = data.frame(t(data))

    expr_mean = data.frame( mean_expr = rowMeans(data))
    colnames(expr_mean) = i
    rownames(expr_mean) = rownames(data)

    final_data = merge(final_data, data, by = 0)
    final_data_vis = merge(final_data_vis, expr_mean, by = 0)

    rownames(final_data) =  final_data$Row.names
    rownames(final_data_vis) = final_data_vis$Row.names
    final_data$Row.names = NULL
    final_data_vis$Row.names = NULL
    }

   

In [286]:
ncol(final_data)

[1] 13172

In [287]:
### Distribution of 0 in final data

In [288]:
sample_data$sample_merge = paste0(sample_data$id, '-', sample_data$library)

In [289]:
#head(sample_data)

In [290]:
dim(final_data)

[1]   121 13172

In [291]:
final_data = merge(final_data, sample_data[,c('sample_id', 'sample_merge')], by.x = 'samples', by.y = 'sample_merge')

In [292]:
dim(final_data)

[1]   121 13173

In [293]:
rownames(final_data)  = final_data$samples

In [294]:
#head(final_data_vis)

In [295]:
dim(final_data)

[1]   121 13173

In [296]:
dim(final_data_vis)

[1] 121  15

#### Filter genes

In [297]:
### Remove mitochondrial & ribosomal genes

In [298]:
ncol(final_data)

[1] 13173

In [299]:
final_data = final_data[, !colnames(final_data) %in% (colnames(final_data)[!is.na(str_extract(colnames(final_data), '__MT.*|__RPL.*|__RPS.*'))])]

In [300]:
ncol(final_data)   

[1] 11833

In [301]:
## Genes with high variance

In [302]:
final_data$samples = NULL

In [303]:
final_data$sample_id = NULL

In [304]:
gene_variance = apply(final_data, 2, function(x) {var( x,na.rm = TRUE)})

In [305]:
head(gene_variance)

X0_T.cell.CD4__SSU72  X0_T.cell.CD4__PARK7   X0_T.cell.CD4__ENO1 
          0.004886796           0.008210134           0.008087536 
 X0_T.cell.CD4__CAPZB X0_T.cell.CD4__HP1BP3  X0_T.cell.CD4__CDC42 
          0.005022516           0.010818624           0.013363703

In [306]:
gene_variance_per_type = data.frame(var = names(gene_variance), type = str_replace(names(gene_variance), '__.*', ''), variance = gene_variance)

In [307]:
head(gene_variance_per_type,2)

,var,type,variance
,<chr>,<chr>,<dbl>
X0_T.cell.CD4__SSU72,X0_T.cell.CD4__SSU72,X0_T.cell.CD4,0.004886796
X0_T.cell.CD4__PARK7,X0_T.cell.CD4__PARK7,X0_T.cell.CD4,0.008210134


In [308]:
var_threshold = quantile(gene_variance, probs = seq(0, 1, 0.01), na.rm = FALSE,
         names = TRUE)['25%']

In [309]:
length(gene_variance)

[1] 11831

In [310]:
var_threshold

25% 
0.02088607

In [311]:
keep_genes = names(gene_variance[gene_variance > var_threshold])

In [312]:
ncol(final_data)

[1] 11831

In [313]:
head(keep_genes)

[1] "X0_T.cell.CD4__CD52"     "X0_T.cell.CD4__SNHG12"  
[3] "X0_T.cell.CD4__MARCKSL1" "X0_T.cell.CD4__SMAP2"   
[5] "X0_T.cell.CD4__PRDX1"    "X0_T.cell.CD4__NASP"

In [314]:
remove_genes = names(gene_variance[gene_variance <= var_threshold])

In [315]:
ncol(final_data)

[1] 11831

In [316]:
final_data$samples = rownames(final_data)

In [317]:
final_data = merge(final_data, sample_data[,c('sample_id', 'sample_merge')], by.x = 'samples', by.y = 'sample_merge')

#### Prepare long format

In [318]:
final_data_long = melt(final_data)

Using samples, sample_id as id variables



In [319]:
### Decide what to do with duplicates

In [320]:
head(final_data_long,2)

,samples,sample_id,variable,value
,<chr>,<chr>,<fct>,<dbl>
1,1.1-L1,m1.1,X0_T.cell.CD4__SSU72,0.5662586
2,10-L11,k10,X0_T.cell.CD4__SSU72,0.5896835


In [321]:
add_duplicates

[1] FALSE

In [322]:
if(add_duplicates == TRUE){
    final_data_long$sample_id[final_data_long$samples == '13.2-L6']  = 'm13.22'      #13.2-L5, 13.2-L6	, 6.4-L10, 6.4-L14	
    final_data_long$sample_id[final_data_long$samples == '6.4-L14']  = 'm6.42'
    }


In [323]:
final_data_long$samples = NULL

In [324]:
final_data_long$type = 'single_cell'

In [325]:
final_data_long = final_data_long %>% group_by(sample_id, type, variable) %>% summarise(value = mean(value))  # take average for samples measured twice

`summarise()` has grouped output by 'sample_id', 'type'. You can override using
the `.groups` argument.


In [326]:
sort(unique(final_data_long$sample_id))

[1] "k10"   "k11"   "k12"   "k13"   "k14"   "k15"   "k16"   "k17"   "k18"  
 [10] "k19"   "k2"    "k20"   "k21"   "k22"   "k23"   "k24"   "k25"   "k26"  
 [19] "k27"   "k28"   "k29"   "k3"    "k30"   "k31"   "k32"   "k33"   "k34"  
 [28] "k4"    "k5"    "k6"    "k7"    "k8"    "k9"    "m1.1"  "m10.2" "m10.3"
 [37] "m11.2" "m11.3" "m11.4" "m12.1" "m12.2" "m13.1" "m13.2" "m13.3" "m13.4"
 [46] "m14.1" "m14.2" "m15.1" "m15.2" "m15.3" "m15.4" "m16.1" "m16.2" "m17.1"
 [55] "m18.1" "m18.2" "m18.3" "m19.1" "m19.2" "m19.3" "m2.1"  "m2.2"  "m2.3" 
 [64] "m2.4"  "m20.3" "m20.4" "m21.1" "m21.2" "m21.3" "m21.4" "m22.1" "m22.2"
 [73] "m22.3" "m22.4" "m23.1" "m23.2" "m23.3" "m23.4" "m24.2" "m24.3" "m24.4"
 [82] "m25.1" "m25.2" "m25.3" "m26.1" "m26.2" "m26.3" "m26.4" "m27.1" "m27.2"
 [91] "m27.3" "m28.2" "m28.3" "m28.4" "m3.2"  "m3.3"  "m3.4"  "m4.1"  "m4.2" 
[100] "m4.3"  "m4.4"  "m5.1"  "m5.3"  "m5.4"  "m6.1"  "m6.2"  "m6.3"  "m6.4" 
[109] "m7.1"  "m7.2"  "m7.3"  "m7.4"  "m8.1"  "m8.2"  "m8.3"  "m8.4"  "m9.1" 
[118] "m9.2"  "m9.3"

In [327]:
length(unique(final_data_long$variable))

[1] 11831

# Integration of all data sources (V1 with gene-gene correletations)

## Combine all data sources

In [328]:
data_long = rbind(final_data_long, cytokines_long,proteomics_long, neutrophils_long, clinical_data_long )
#data_long = rbind(final_data_long, cytokines_long,proteomics_long, neutrophils_long ) # vesion without clinical data

In [329]:
length(unique(final_data_long$variable))

[1] 11831

In [330]:
head(data_long,2)

sample_id,type,variable,value
<chr>,<chr>,<chr>,<dbl>
k10,single_cell,X0_T.cell.CD4__SSU72,0.5896835
k10,single_cell,X0_T.cell.CD4__PARK7,0.8587511


In [331]:
length(unique(data_long$variable))

[1] 13289

In [332]:
unique(data_long$type)

[1] "single_cell"   "cytokine"      "proteomics"    "neutrophil"   
[5] "clinical_data"

In [333]:
data_long$config = paste0(quantile_normalization_cyto, '-', quantile_normalization_proteomics, '-', quantile_normalization_neutrophils, '-', neutrophil_threshold, '-', regress_neutrophils, '-', library_adjustment_neutrophils, '-')

In [334]:
write.csv(data_long, paste0(result_path, '/E-Analysis/Combined_Data_', name, '.csv'))